In [10]:
import os
import sys
from pathlib import Path
# Setting the working directory to the root of the project
project_dir = Path("C:/Users/adbou/source/repos/KFHXRelatedAi/")
os.chdir(project_dir)

from Configs.GeneralPaths import SOURCEDATA
import pandas as pd
import numpy as np
import ast


In [11]:
#load user transactions data
deals_embeddings = pd.read_csv(Path(SOURCEDATA / "Deals_Embeddings.csv"))
deals_embeddings['ada_embedding'] = deals_embeddings['ada_embedding'].apply(ast.literal_eval)



In [12]:
deals_embeddings.head()

,ContentId,ada_embedding
0,113815,"[0.0036637482699006796, 0.0040904744528234005,..."
1,113816,"[0.006569046527147293, 0.006756171118468046, -..."
2,113817,"[0.004525594413280487, -0.03140182048082352, 0..."
3,113818,"[-0.003238065168261528, -0.08213862031698227, ..."
4,113819,"[-0.05140183866024017, -0.05169753357768059, 0..."


In [13]:
#loading user transactions
user_transactions = pd.read_excel(Path(SOURCEDATA / "Transaction_User.xlsx"))
user_tranactions = user_transactions.drop(columns=['TrxId'])
user_tranactions.head()

,FK_BusinessUserId,PointsRedeemed,FK_ContentId
0,976480,1000,113923
1,976480,10000,113853
2,976480,10000,113853
3,976921,10,113851
4,976480,50,113835


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(embedding, embeddings):
    similarities = cosine_similarity([embedding], embeddings)
    return similarities[0]


In [18]:
def recommend_deals_for_user(user_id, user_deals_df, deals_embeddings_new):
    # Get the user's redeemed deals' content IDs
    redeemed_content_ids = user_deals_df[user_deals_df['FK_BusinessUserId'] == user_id]['FK_ContentId'].unique()
    
    # Retrieve the embeddings for the redeemed deals
    redeemed_embeddings = deals_embeddings_new[deals_embeddings_new['ContentId'].isin(redeemed_content_ids)]
    
    if redeemed_embeddings.empty:
        print("No redeemed deals found for this user.")
        return pd.DataFrame()
    
    # Get all embeddings
    all_embeddings = deals_embeddings_new['ada_embedding'].tolist()

    # Initialize an empty list to store similarities
    similarities = []

    # Calculate similarities for each redeemed embedding
    for embedding in redeemed_embeddings['ada_embedding']:
        similarity = calculate_cosine_similarity(embedding, all_embeddings)
        similarities.append(similarity)

    # Average the similarities to get a single similarity score for each deal
    average_similarities = np.mean(similarities, axis=0)

    # Add the similarities to the DataFrame
    deals_embeddings_new['similarity'] = average_similarities

    # Filter out deals the user has already redeemed
    recommended_deals = deals_embeddings_new[~deals_embeddings_new['ContentId'].isin(redeemed_content_ids)]

    # Sort by similarity in descending order and take the top 5 recommendations
    top_5_recommendations = recommended_deals.sort_values(by='similarity', ascending=False).head(5)

    return top_5_recommendations




In [25]:
user_id = 985749

# Get recommendations for the user
top_5_recommendations = recommend_deals_for_user(user_id, user_tranactions, deals_embeddings)

In [26]:
top_5_recommendations

,ContentId,ada_embedding,similarity
4,113819,"[-0.05140183866024017, -0.05169753357768059, 0...",0.695060
48,113863,"[-0.03503214195370674, -0.030069053173065186, ...",0.678203
101,113916,"[-0.0008215493289753795, -0.01800406165421009,...",0.626454
149,115104,"[-0.021717684343457222, -0.04306768625974655, ...",0.608945
198,115244,"[0.007270508445799351, -0.017816953361034393, ...",0.606802


In [27]:
user_trx = user_tranactions[user_tranactions['FK_BusinessUserId'] == 985749]
user_trx

,FK_BusinessUserId,PointsRedeemed,FK_ContentId
12596,985749,10,113851
12610,985749,5000,113864
